In [ ]:
using Revise, FiniteDifferenceMethods

# Mesh

In [ ]:
const T = Float64

n = 513 # number of unknowns (2 ^ p + 1 for exact arithmetics)
o = 1   # width of ghost layer

inds = range(1+o, n+o)

@assert o ≥ 1

In [ ]:
x = collocated(n+2o, T, start=first(inds), stop=last(inds))
xm = staggered(n+2o, T, start=first(inds), stop=last(inds))

h = step(x)

@assert isequal(step(x), step(xm))

# Apertures

Let $a \equiv 0$, $b = 1$ and $s \in \left ( a, b \right )$ (the interface position). Assume fluid domain occupies
$$
\Omega \equiv \left ( s, b \right ).
$$

In [ ]:
s = x[o+1] + h / 4 # interface position

@assert x[o+1] < s < xm[o+1] < x[o+2]

In [ ]:
# implicit boundary (depends on s or level set)

A = [T(el > s) for el in x]
B = [T(el > s) for el in xm]
W = [max(min(el+h/2-s, h), 0) for el in x];

![Case `n = 9` and `o = 2`.](definition.png "File definition.png missing.")

In [ ]:
# explicit boundary (hard-coded)

inds = eachindex(A, B, W)[begin+o:begin+o+n-1]

A[last(inds):end] .= 0
B[last(inds):end] .= 0
W[last(inds)+1:end] .= 0

W[last(inds)] = h / 2;

# Operators

In [ ]:
row = col = CartesianIndices((inds,))

Gω = Gradient{:x,:ω}((A, B, W), row, col)
Gγ = Gradient{:x,:γ}((A, B, W), row, col);

In [ ]:
getindex(Gω, 2:4, 1:5)

In [ ]:
Dω = Divergence{:x,:ω}((A, B), row, col)
Dγ = Divergence{:x,:γ}((A, B), row, col);

In [ ]:
getindex(Dω, 2:4, 1:5)

# CSC format

In [ ]:
using SparseArrays, BenchmarkTools

In [ ]:
spGω = sparse(Gω)
spGγ = sparse(Gγ);

In [ ]:
@btime collect($Gω)
@btime sparse($Gω);

In [ ]:
@btime (^)($spGω, 2)

# Test correctness

Works for arbitrary geometric moments...

In [ ]:
rA = rand(length(x))
rB = rand(length(xm))
rW = rand(length(x))

rGω = Gradient{:x,:ω}((rA, rB, rW), row, col)
rGγ = Gradient{:x,:ω}((rA, rB, rW), row, col)

@assert iszero(sparse(rGω) - rGω)
@assert iszero(sparse(rGγ) - rGγ)

... and possible configurations.

In [ ]:
for _ in 1:10_000
    row = CartesianIndices((rand(inds):rand(inds),))
    col = CartesianIndices((rand(inds):rand(inds),))

    rGω = Gradient{:x,:ω}((rA, rB, rW), row, col)

    @assert iszero(sparse(rGω) - collect(rGω))
end

In [ ]:
(row, col) = (CartesianIndices((5:6,)), CartesianIndices((3:4,)))
rGω = Gradient{:x,:ω}((rA, rB, rW), row, col)
sparse(rGω)

In [ ]:
(row, col) = CartesianIndices((3:4,)), (CartesianIndices((4:5,)))
rGω = Gradient{:x,:ω}((rA, rB, rW), row, col)
sparse(rGω)